In [1]:
# import pandas as pd
# from varseek.utils import explode_df

# mutation_metadata_df_path = "/home/jrich/data/varseek_data_fresh/vk_build_pipeline_grch37_ensembl93/mutation_metadata_df_updated_vk_info.csv"
# mutation_metadata_df_out_path_exploded = "/home/jrich/data/varseek_data_fresh/vk_build_pipeline_grch37_ensembl93/mutation_metadata_df_updated_vk_info_exploded.csv"
# columns_to_explode = ["header","order","seq_ID","mutation","mutation_type","wt_sequence","nucleotide_positions","actual_mutation","mutation_cds","mutation_aa","GENOMIC_MUTATION_ID","chromosome","strand","mutation_genome","gene_name","mutation_id","start_mutation_position","end_mutation_position","nucleotide_positions_cdna","actual_mutation_cdna","start_mutation_position_cdna","end_mutation_position_cdna","header_genome","header_cdna","header_cds","nucleotide_positions_genome","actual_mutation_genome","start_mutation_position_genome","end_mutation_position_genome","distance_to_nearest_splice_junction","is_near_splice_junction_10","nearby_mutations","nearby_mutations_count","has_a_nearby_mutation"]

# mutation_metadata_df = pd.read_csv(mutation_metadata_df_path)
# mutation_metadata_df_exploded = explode_df(mutation_metadata_df, columns_to_explode)
# mutation_metadata_df_exploded.to_csv(mutation_metadata_df_out_path_exploded, index=False)

In [2]:
import random
import varseek as vk
import os
import sys
import anndata as ad
import subprocess
import numpy as np
import pandas as pd

from varseek.utils import build_random_genome_read_df, fasta_to_fastq, create_mutant_t2g, fasta_to_fastq, find_genes_with_aligned_reads_for_kb_extract, calculate_metrics, create_stratified_metric_bar_plot, create_venn_diagram, get_header_set_from_fastq, plot_histogram, synthetic_data_summary_plot, plot_basic_bar_plot_from_dict, draw_confusion_matrix, check_for_read_kmer_in_mcrs

In [3]:
read_length = 150
strand = None  # None for strand-agnostic (randomly-selected), "f" for forward, "r" for reverse, "both" for both - make sure this matches the reference genome (kropp build command) - strand = True -> "f" or "r" here; strand = False -> None or "both" here - note that the strand is randomly selected per *transcript*, such that all drawn reads will come from the same strand no matter what
add_noise=False
error_rate=0
max_errors=0
seq_id_column="seq_ID"
mut_column="mutation"
threads = 32


dlist_kb_argument = "None"  # path to dlist fasta file or "None" (including the quotes)
k = 55

run_kb_extract_slow = True
run_kb_extract_in_notebook = True
number_of_parallel_processes = 32  # used iff run_kb_extract_in_notebook == True
chunk_size = 3000  # used iff run_kb_extract_in_notebook == True




# Paths
out_dir_notebook="/home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp"

#!!!CHANGE TO GRCH37 (commented)
mutation_metadata_df = "/home/jrich/data/varseek_data_fresh/vk_build_pipeline_t2t/mutation_metadata_df_updated_vk_info_exploded.csv" # "/home/jrich/data/varseek_data_fresh/vk_build_pipeline_grch37_ensembl93/mutation_metadata_df_updated_vk_info_exploded.csv"
#!!!CHANGE TO GRCH37 (commented)

sequences = "/home/jrich/data/varseek_data_fresh/reference/ensembl_grch37_release93/Homo_sapiens.GRCh37.cdna.all.fa"
reference_out_dir = "/home/jrich/data/varseek_data_fresh/reference"
mutation_index = "/home/jrich/data/varseek_data_fresh/vk_build_pipeline_grch37_ensembl93/mutation_reference.idx"
mutation_t2g = "/home/jrich/data/varseek_data_fresh/vk_build_pipeline_grch37_ensembl93/t2g_filtered.txt"

In [4]:
synthetic_reads_fastQ = f"{out_dir_notebook}/synthetic_reads.fq"
out_dir_vk_build = f"{out_dir_notebook}/vk_build"

kb_count_out = f"{out_dir_notebook}/kb_count_out"
kb_extract_out_total = f"{out_dir_notebook}/kb_extract_out_total"
kb_extract_out_total_mm = f"{out_dir_notebook}/kb_extract_out_total_mm"
kb_extract_out_all_slow = f"{out_dir_notebook}/kb_extract_out_all_slow"

os.makedirs(out_dir_notebook, exist_ok=True)

if strand is None or strand == "both":
    strand_kb_extract = "unstranded"
elif strand == "f":
    strand_kb_extract = "forward"
elif strand == "r":
    strand_kb_extract = "reverse"

mutation_metadata_df_out = mutation_metadata_df.replace(".csv", "_with_synthetic_read_info.csv")
read_df_out = synthetic_reads_fastQ.replace(".fq", ".csv")

seed=42
read_df = None
mutation_metadata_df_path = mutation_metadata_df
adata_path = f"{kb_count_out}/counts_unfiltered/adata.h5ad"

In [5]:
# # run notebook 1 up through kb ref, or run the following commands
# vk build
# vk info with save_exploded_df=True
# vk filter
# kb ref

# Simple synthetic reads - erase

In [6]:
#!!! TEMP
import varseek
mutation_metadata_df_path = mutation_metadata_df
mutation_metadata_df = pd.read_csv(mutation_metadata_df_path)

sim_data_df_path = "/home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp/vk_build/sim_data_df.csv"
sim_data_df = pd.read_csv(sim_data_df_path, usecols=["header", "mutant_sequence", "wt_sequence"])

sim_data_df.rename(
    columns={
        "mutant_sequence": "mutant_sequence_read_parent",
        "wt_sequence": "wt_sequence_read_parent",
    },
    inplace=True,
)

sim_data_df["mutant_sequence_read_parent_rc"] = sim_data_df["mutant_sequence_read_parent"].apply(varseek.varseek_build.reverse_complement)
sim_data_df["mutant_sequence_read_parent_length"] = sim_data_df["mutant_sequence_read_parent"].str.len()

sim_data_df["wt_sequence_read_parent_rc"] = sim_data_df["wt_sequence_read_parent"].apply(varseek.varseek_build.reverse_complement)
sim_data_df["wt_sequence_read_parent_length"] = sim_data_df["wt_sequence_read_parent"].str.len()

mutation_metadata_df = pd.merge(
    mutation_metadata_df,
    sim_data_df[
        [
            "header",
            "mutant_sequence_read_parent",
            "mutant_sequence_read_parent_rc",
            "mutant_sequence_read_parent_length",
            "wt_sequence_read_parent",
            "wt_sequence_read_parent_rc",
            "wt_sequence_read_parent_length",
        ]
    ],
    on="header",
    how="left",
    suffixes=("", "_read_parent"),
)

/tmp/ipykernel_134811/3237759037.py:4: DtypeWarning: Columns (12,52,67,69) have mixed types. Specify dtype option on import or set low_memory=False.
  mutation_metadata_df = pd.read_csv(mutation_metadata_df_path)


In [9]:
import sys
import importlib

# Remove the module from sys.modules
if 'varseek.sim' in sys.modules:
    del sys.modules['varseek.sim']

# Import the module and reload it
import varseek.varseek_sim
importlib.reload(varseek.varseek_sim)
import varseek as vk

# instead of using vk.sim() as before, I must explicitely use vk.varseek_sim.sim()

In [10]:
#!!! TEMP
read_fq_path_specific = f"{out_dir_notebook}/synthetic_reads.fq"

conditions = ['dlist-equal=none', 'kmer_overlap_in_mcrs_reference-isfalse']


if not os.path.exists(mutation_metadata_df_out) and not os.path.exists(read_df_out):
    simulated_df_dict = vk.varseek_sim.sim(
        mutation_metadata_df = mutation_metadata_df,
        fastq_output_path = synthetic_reads_fastQ,
        sample_type="m",
        number_of_mutations_to_sample=200,
        strand=strand,
        number_of_reads_per_sample="all",  # not used when number_of_reads_per_sample_m and number_of_reads_per_sample_w are provided
        read_length=read_length,
        seed=seed,
        add_noise=add_noise,
        error_rate=error_rate,
        max_errors=max_errors,
        with_replacement=False,
        sequences=sequences,
        mutation_metadata_df_path=mutation_metadata_df_path,
        seq_id_column=seq_id_column,
        mut_column=mut_column,
        reference_out_dir=reference_out_dir,
        out_dir_vk_build=out_dir_vk_build,
        filters=conditions,
        read_df_out=read_df_out,
        mutation_metadata_df_out=mutation_metadata_df_out,
    )

    read_df, mutation_metadata_df = simulated_df_dict["read_df"], simulated_df_dict["mutation_metadata_df"]
else:
    read_df = pd.read_csv(read_df_out)
    mutation_metadata_df = pd.read_csv(mutation_metadata_df_out)

15:00:57 - INFO - Initial number of mutations: 5344110
15:01:24 - INFO - Filtered 60168 mutations dlist with equal none - 5283942 mutations remaining
15:01:31 - INFO - Filtered 160733 mutations kmer_overlap_in_mcrs_reference with isfalse False - 5123209 mutations remaining
15:01:55 - INFO - Output fasta file with filtered mutations: None
15:01:55 - INFO - t2g file containing mutated sequences created at None.


Error: 'NoneType' object has no attribute 'composition'
Wrote 59197 mutations to /home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp/synthetic_reads.fq


# Complex synthetic reads - uncomment

In [ ]:
# number_of_mutations_to_sample = 100
# m_list = [1, 2, 3, 4, 5, 6, 7, 8, 16, 32, 64, 128, 256, 512]
# w_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 16, 32, 64, 128, 256, 512]
# conditions = ['mutation_type-equal=substitution', 'mutation_type-notequal=substitution', 'dlist-equal=none', 'dlist-notequal=none', 'kmer_overlap_in_mcrs_reference-isfalse', 'kmer_overlap_in_mcrs_reference-istrue', 'is_near_splice_junction_10-isfalse', 'is_near_splice_junction_10-istrue']

# random.shuffle(m_list)
# random.shuffle(w_list)

# if not os.path.exists(mutation_metadata_df_out) and not os.path.exists(read_df_out):
#     for m in m_list:
#         for w in w_list:
#             for condition in conditions:
#                 read_fq_path_specific = synthetic_reads_fastQ.replace(".fq", f"_m{m}_w{w}_{condition.replace('=', '_')}.fq")
#                 condition_added_list = [condition, 'included_in_synthetic_reads-isnottrue']
                
#                 simulated_df_dict = vk.sim(
#                     mutation_metadata_df = mutation_metadata_df,
#                     fastq_output_path = read_fq_path_specific,
#                     fastq_parent_path = synthetic_reads_fastQ,
#                     read_df_parent=read_df,
#                     sample_type="all",
#                     number_of_mutations_to_sample=number_of_mutations_to_sample,
#                     strand=strand,
#                     number_of_reads_per_sample=None,  # not used when number_of_reads_per_sample_m and number_of_reads_per_sample_w are provided
#                     number_of_reads_per_sample_m=m,
#                     number_of_reads_per_sample_w=w,
#                     read_length=read_length,
#                     seed=seed,
#                     add_noise=add_noise,
#                     error_rate=error_rate,
#                     max_errors=max_errors,
#                     with_replacement=False,
#                     filters=condition_added_list,
#                     sequences=sequences,
#                     mutation_metadata_df_path=mutation_metadata_df_path,
#                     seq_id_column=seq_id_column,
#                     mut_column=mut_column,
#                     reference_out_dir=reference_out_dir,
#                     out_dir_vk_build=out_dir_vk_build,
#                 )

#                 mutation_metadata_df, read_df = simulated_df_dict['mutation_metadata_df'], simulated_df_dict['read_df']
# else:
#     read_df = pd.read_csv(read_df_out)
#     mutation_metadata_df = pd.read_csv(mutation_metadata_df_out)

# Random genome - uncomment

In [ ]:
# # do kv sim with random genome
# mutation_metadata_df['start_position_for_which_read_contains_mutation_cdna'] = mutation_metadata_df['start_mutation_position_cdna'] - read_length + 1
# mutation_metadata_df['start_position_for_which_read_contains_mutation_genome'] = mutation_metadata_df['start_mutation_position_genome'] - read_length + 1

# number_of_random_reads_cdna = 10
# number_of_random_reads_genome = 10

# # read_df = build_random_genome_read_df(reference_fasta_file_path = reference_cdna_fasta, mutation_metadata_df = mutation_metadata_df, read_df = read_df, n = number_of_random_reads_cdna, read_length = read_length, input_type = "transcriptome", strand = None)
# # read_df = build_random_genome_read_df(reference_fasta_file_path = reference_genome_fasta, mutation_metadata_df = mutation_metadata_df, read_df = read_df, n = number_of_random_reads_genome, read_length = read_length, input_type = "genome", strand = None)

In [21]:
# kb count
if not os.path.exists(kb_count_out) or len(os.listdir(kb_count_out)) == 0:
    #!!! add "--num", if this stays around
    kb_count_command = ["kb", "count", "-t", str(threads), "-k", str(k), "-i", mutation_index, "-g", mutation_t2g, "-x", "bulk", "--h5ad", "--parity", "single", "-o", kb_count_out, synthetic_reads_fastQ]
    subprocess.run(kb_count_command, check=True)

[2024-11-08 15:15:00,944]    INFO [count] Using index /home/jrich/data/varseek_data_fresh/vk_build_pipeline_grch37_ensembl93/mutation_reference.idx to generate BUS file to /home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp/kb_count_out from
[2024-11-08 15:15:00,944]    INFO [count]         /home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp/synthetic_reads.fq
[2024-11-08 15:15:42,291]    INFO [count] Sorting BUS file /home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp/kb_count_out/output.bus to /home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp/kb_count_out/tmp/output.s.bus
[2024-11-08 15:15:45,299]    INFO [count] Inspecting BUS file /home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp/kb_count_out/tmp/output.s.bus
[2024-11-08 15:15:46,405]    INFO [count] Generating count matrix /home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp/kb_count_out/counts_unfiltered/cells_x_genes from BUS file /home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp/kb

Lay out kb extract commands

In [22]:
# run kb extract script
if not os.path.exists(kb_extract_out_total):
    kb_extract_crude_command = f"kb extract --extract_all_fast --strand {strand_kb_extract} --verbose -k {k} -t {threads} -o {kb_extract_out_total} -i {mutation_index} -g {mutation_t2g} {synthetic_reads_fastQ}"
    if run_kb_extract_in_notebook:
        try:
            result = subprocess.run(kb_extract_crude_command, shell=True, check=True, text=True)
        except subprocess.CalledProcessError as e:
            print("An error occurred:")
            print(e)
            print("Standard Error Output:")
            print(e.stderr)
    else:
        print(kb_extract_crude_command)

if not os.path.exists(kb_extract_out_total_mm):
    kb_extract_crude_opposite_mm_command = f"kb extract --extract_all_fast --strand {strand_kb_extract} --mm --verbose -k {k} -t {threads} -o {kb_extract_out_total_mm} -i {mutation_index} -g {mutation_t2g} {synthetic_reads_fastQ}"
    if run_kb_extract_in_notebook:
        try:
            result = subprocess.run(kb_extract_crude_opposite_mm_command, shell=True, check=True, text=True)
        except subprocess.CalledProcessError as e:
            print("An error occurred:")
            print(e)
            print("Standard Error Output:")
            print(e.stderr)
    else:
        print(kb_extract_crude_opposite_mm_command)


if run_kb_extract_slow and not os.path.exists(kb_extract_out_all_slow):
    # TODO: ensure that this is the same as listing each MCRS individually
    kb_extract_command = f"kb extract --strand {strand_kb_extract} --verbose -k {k} --mm -t {threads} --extract_all -o {kb_extract_out_all_slow} -i {mutation_index} -g {mutation_t2g} {synthetic_reads_fastQ}"
    # # removed {multimapped_line} in favor of always doing --mm because I can just filter out multimapped reads later
    # # can replace --targets {mcrs_string_with_aligned_reads} with --extract_all for the same final effect, but it will output all files at once rather than one at a time (not good if I want to check progress throughout or break up for multithreading, but it can avoid the issue where mcrs_string_with_aligned_reads is too many characters)
    # # adata_path = f"{kb_count_out}/counts_unfiltered/adata.h5ad"
    # # mapped_mutations_string = find_genes_with_aligned_reads_for_kb_extract(adata_path)
    # # mcrs_set_with_aligned_reads = set(mapped_mutations_string.split())
    # mcrs_string_with_aligned_reads = ' '.join(f"'{x}'" for x in mcrs_set_with_aligned_reads)
    # kb_extract_command = f"kb extract --verbose -k {k} --mm -t {threads} --targets {mcrs_string_with_aligned_reads} -o {kb_extract_out_all_slow} -i {mutation_index} -g {mutation_t2g} {synthetic_reads_fastQ}"
    if run_kb_extract_in_notebook:
        try:
            result = subprocess.run(kb_extract_command, shell=True, check=True, text=True)
        except subprocess.CalledProcessError as e:
            print("An error occurred:")
            print(e)
            print("Standard Error Output:")
            print(e.stderr)
    else:
        print(kb_extract_command)

if not run_kb_extract_in_notebook:    
    print("Run the following command in tmux to extract reads in a multithreaded fashion:")
    print(f"python3 /home/jrich/Desktop/CART_prostate_sc/src/run_kb_extract.py -k {k} -t {threads} --strand {strand_kb_extract} --parallel {number_of_parallel_processes} --chunk {chunk_size} --adata {adata_path} --fastq {synthetic_reads_fastQ} -i {mutation_index} -g {mutation_t2g} --no-mm-fast-out {kb_extract_out_total} --mm-fast-out {kb_extract_out_total_mm} --slow-out {kb_extract_out_all_slow}")

[2024-11-08 15:16:55,562]   DEBUG [main] Printing verbose output
[2024-11-08 15:16:57,767]   DEBUG [main] kallisto binary located at /home/jrich/miniconda3/envs/varseek/lib/python3.10/site-packages/kb_python/bins/linux/kallisto/kallisto_k64
[2024-11-08 15:16:57,767]   DEBUG [main] bustools binary located at /home/jrich/miniconda3/envs/varseek/lib/python3.10/site-packages/kb_python/bins/linux/bustools/bustools
[2024-11-08 15:16:57,767]   DEBUG [main] Creating `/home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp/kb_extract_out_total/tmp` directory
[2024-11-08 15:16:57,773]   DEBUG [main] Namespace(list=False, command='extract', tmp=None, keep_tmp=False, verbose=True, fastq='/home/jrich/data/varseek_data_fresh/vk_sim_2024nov24_temp/synthetic_reads.fq', i='/home/jrich/data/varseek_data_fresh/vk_build_pipeline_grch37_ensembl93/mutation_reference.idx', targets=None, target_type='gene', extract_all=False, extract_all_fast=True, extract_all_unmapped=False, mm=False, g='/home/jrich/data/v

In [ ]:
if (not os.path.exists(kb_extract_out_total) or (run_kb_extract_slow and not os.path.exists(kb_extract_out_all_slow))):
    raise Exception("kb extract has not been run. Please run it before continuing.")

# MY BUS MY BUS

In [ ]:
from varseek.utils import make_bus_df
kb_count_out_mm = f"{out_dir_notebook}/kb_count_out_mm"
if not os.path.exists(kb_count_out_mm) or len(os.listdir(kb_count_out_mm)) == 0:
    kb_count_command = ["kb", "count", "-t", str(threads), "--mm", "-k", str(k), "-i", mutation_index, "-g", mutation_t2g, "-x", "bulk", "--num", "--h5ad", "--parity", "single", "-o", kb_count_out_mm, synthetic_reads_fastQ]
    subprocess.run(kb_count_command, check=True)

bus_df = make_bus_df(kallisto_out = kb_count_out_mm, fastq_file = synthetic_reads_fastQ, t2g_file = mutation_t2g, mm = True, union = False, assay = "bulk", bustools = "/home/jrich/miniconda3/envs/varseek/lib/python3.10/site-packages/kb_python/bins/linux/bustools/bustools")

# TODO: edit this below - mcrs_df is output of vk info, and mutation_metadata_df is exploded output of vk_info

In [3]:
# TODO: Add columns to mcrs_df for any_noisy_reads and all_noisy_reads

In [ ]:
adata_path = f"{kb_count_out}/counts_unfiltered/adata.h5ad"
adata = ad.read_h5ad(adata_path)

mapped_mutations_string = find_genes_with_aligned_reads_for_kb_extract(adata_path)

mcrs_set_with_aligned_reads = set(mapped_mutations_string.split())

# Find the indices of non-zero values in adata.X
non_zero_indices = np.nonzero(adata.X)

# Get the non-zero values
non_zero_values = np.squeeze(np.asarray(adata.X[non_zero_indices])).astype(int)

# Get the corresponding gene names
mcrs_headers = adata.var.index[non_zero_indices[1]].to_numpy().flatten()


# Create a DataFrame
result_df = pd.DataFrame({
    'mcrs_header': mcrs_headers,
    'number_of_reads_aligned_to_this_item': non_zero_values
})

mutation_metadata_df = mutation_metadata_df.merge(result_df, on='mcrs_header', how='left')
mutation_metadata_df['received_an_aligned_read'] = ~pd.isna(mutation_metadata_df['number_of_reads_aligned_to_this_item'])

# # set to int - will need to do Int64 to work
# mutation_metadata_df['number_of_reads_mutant'] = mutation_metadata_df['number_of_reads_mutant'].astype(int)
# mutation_metadata_df['number_of_reads_wt'] = mutation_metadata_df['number_of_reads_wt'].astype(int)
# mutation_metadata_df['number_of_reads_aligned_to_this_item'] = mutation_metadata_df['number_of_reads_aligned_to_this_item'].astype(int)

In [ ]:
def concatenate_value_to_list(header, indices_list):
    return [f"{header}_{index}" for index in indices_list]

mutation_metadata_df.loc[(mutation_metadata_df['concatenated_headers_in_mcrs'] == False) | (pd.isna(mutation_metadata_df['list_of_read_starting_indices_mutant'])), 'list_of_read_starting_indices_mutant_copy'] = mutation_metadata_df.loc[(mutation_metadata_df['concatenated_headers_in_mcrs'] == False) | (pd.isna(mutation_metadata_df['list_of_read_starting_indices_mutant'])), 'list_of_read_starting_indices_mutant']
mutation_metadata_df.loc[(mutation_metadata_df['concatenated_headers_in_mcrs'] == True) & (~pd.isna(mutation_metadata_df['list_of_read_starting_indices_mutant'])), 'list_of_read_starting_indices_mutant_copy'] = mutation_metadata_df.loc[(mutation_metadata_df['concatenated_headers_in_mcrs'] == True) & (~pd.isna(mutation_metadata_df['list_of_read_starting_indices_mutant']))].apply(lambda row: concatenate_value_to_list(row['header'], row['list_of_read_starting_indices_mutant']), axis=1)


mutation_metadata_df.loc[(mutation_metadata_df['concatenated_headers_in_mcrs'] == False) | (pd.isna(mutation_metadata_df['list_of_read_starting_indices_wt'])), 'list_of_read_starting_indices_wt_copy'] = mutation_metadata_df.loc[(mutation_metadata_df['concatenated_headers_in_mcrs'] == False) | (pd.isna(mutation_metadata_df['list_of_read_starting_indices_wt'])), 'list_of_read_starting_indices_wt']
mutation_metadata_df.loc[(mutation_metadata_df['concatenated_headers_in_mcrs'] == True) & (~pd.isna(mutation_metadata_df['list_of_read_starting_indices_wt'])), 'list_of_read_starting_indices_wt_copy'] = mutation_metadata_df.loc[(mutation_metadata_df['concatenated_headers_in_mcrs'] == True) & (~pd.isna(mutation_metadata_df['list_of_read_starting_indices_wt']))].apply(lambda row: concatenate_value_to_list(row['header'], row['list_of_read_starting_indices_wt']), axis=1)


agg_dict = {
    'included_in_synthetic_reads_mutant': 'any',  # OR across rows
    'included_in_synthetic_reads_wt': 'any',      # OR across rows
    'number_of_reads_wt': 'sum',                  # Sum across rows
    'number_of_reads_mutant': 'sum'               # Sum across rows
}

for col in mutation_metadata_df.columns:
    if col not in agg_dict and col != 'mcrs_header':  # Skip 'mcrs_header'
        agg_dict[col] = 'first'

unique_mcrs_df_original_columns = mutation_metadata_df.columns
unique_mcrs_df = mutation_metadata_df.groupby('mcrs_header').agg(agg_dict).reset_index()
unique_mcrs_df = unique_mcrs_df[unique_mcrs_df_original_columns]

unique_mcrs_df['tumor_purity'] = unique_mcrs_df['number_of_reads_mutant'] / unique_mcrs_df['number_of_reads_wt']

unique_mcrs_df['tumor_purity'] = np.where(
    np.isnan(unique_mcrs_df['tumor_purity']), 
    np.nan,  # Keep NaN as NaN
    unique_mcrs_df['tumor_purity']  # Keep the result for valid divisions
)

unique_mcrs_df = unique_mcrs_df[['mcrs_header', 'mcrs_sequence', 'mcrs_mutation_type', 'mcrs_sequence_length', 'dlist', 'number_of_alignments_to_normal_human_reference_cdna', 'number_of_alignments_to_normal_human_reference_genome', 'number_of_kmers_with_overlap_to_other_mcrs_items_in_mcrs_reference', 'number_of_mcrs_items_with_overlapping_kmers_in_mcrs_reference', 'kmer_overlap_in_mcrs_reference', 'distance_to_nearest_splice_junction', 'is_near_splice_junction', 'included_in_synthetic_reads_wt', 'included_in_synthetic_reads_mutant', 'received_an_aligned_read', 'number_of_reads_wt', 'number_of_reads_mutant', 'tumor_purity', 'list_of_read_starting_indices_wt_copy', 'list_of_read_starting_indices_mutant_copy', 'number_of_reads_aligned_to_this_item', 'concatenated_headers_in_mcrs']]
unique_mcrs_df.rename(columns={'list_of_read_starting_indices_wt_copy': 'list_of_read_starting_indices_wt', 'list_of_read_starting_indices_mutant_copy': 'list_of_read_starting_indices_mutant'}, inplace=True)
unique_mcrs_df_included_in_synthetic_reads = unique_mcrs_df.loc[unique_mcrs_df['included_in_synthetic_reads_wt'] | unique_mcrs_df['included_in_synthetic_reads_mutant']]

In [ ]:
for unique_read_df_column in ["region_included_in_mcrs_reference"]:
    synthetic_data_summary_plot(read_df, unique_read_df_column, out_path = f"{plot_output_folder}/synthetic_data_summary_{unique_read_df_column}.png")

for unique_mcrs_df_column in ["mcrs_mutation_type", "distance_to_nearest_splice_junction", "number_of_kmers_with_overlap_to_other_mcrs_items_in_mcrs_reference", "number_of_mcrs_items_with_overlapping_kmers_in_mcrs_reference", "number_of_alignments_to_normal_human_reference_cdna", "number_of_alignments_to_normal_human_reference_genome", "dlist", "number_of_reads_mutant", "number_of_reads_wt", "tumor_purity"]:
    synthetic_data_summary_plot(unique_mcrs_df_included_in_synthetic_reads, unique_mcrs_df_column, out_path = f"{plot_output_folder}/synthetic_data_summary_{unique_mcrs_df_column}.png")

## Perspective of reads

Big picture - especially useful when only mutant reads (tells us FNs)

In [ ]:
!cat {kb_count_out}/run_info.json | grep -E '"n_targets"|"n_processed"|"n_pseudoaligned"|"n_unique"|"p_pseudoaligned"|"p_unique"'

with open(f"{kb_count_out}/run_info.json", 'r') as f:
    data = json.load(f)

total_reads = data.get("n_processed", None)
n_pseudoaligned = data.get("n_pseudoaligned", None)
n_unique = data.get("n_unique", None)
n_multimapped = n_pseudoaligned - n_unique

if check_assertions:
    assert total_reads == n_pseudoaligned, "Total reads and pseudoaligned reads do not match"
    assert total_reads == n_unique, "Total reads and unique reads do not match"

In [ ]:

kb_extract_all_fastq = f"{kb_extract_out_total}/all/1.fastq.gz"
aligned_reads_kb_extract = get_header_set_from_fastq(kb_extract_all_fastq)

read_df['aligned_somewhere'] = read_df['read_header'].isin(aligned_reads_kb_extract)
read_df['did_not_align_anywhere'] = ~read_df['aligned_somewhere']

read_df['TP'] = (read_df['mutant_read'] & read_df['aligned_somewhere'])
read_df['FP'] = (~read_df['mutant_read'] & read_df['aligned_somewhere'])
read_df['FN'] = (read_df['mutant_read'] & ~read_df['aligned_somewhere'])
read_df['TN'] = (~read_df['mutant_read'] & ~read_df['aligned_somewhere'])

metric_dictionary_reads = calculate_metrics(read_df, header_name = "read_header", check_assertions = check_assertions, out = f"{plot_output_folder}/reads_metrics_crude.txt")
draw_confusion_matrix(metric_dictionary_reads)

In [ ]:
true_set = set(read_df.loc[read_df['mutant_read'], 'read_header'])
positive_set = set(read_df.loc[read_df['aligned_somewhere'], 'read_header'])
create_venn_diagram(true_set, positive_set, TN = metric_dictionary_reads['TN'], mm = n_multimapped, out_path = f"{plot_output_folder}/venn_diagram_read_crude.png")

In [ ]:
kb_extract_all_fastq_opposite_mm = f"{kb_extract_out_total_opposite_mm}/all/1.fastq.gz"
aligned_reads_kb_extract_opposite_mm = get_header_set_from_fastq(kb_extract_all_fastq_opposite_mm)

multimapped_reads = aligned_reads_kb_extract.symmetric_difference(aligned_reads_kb_extract_opposite_mm)
read_df['multimapped'] = read_df['read_header'].isin(multimapped_reads)
print(f"Multimapped reads ({n_multimapped}): {multimapped_reads}")

In [ ]:
read_df = check_for_read_kmer_in_mcrs(read_df, unique_mcrs_df, k, strand = strandedness)

In [ ]:
read_df_fn = read_df[read_df['FN']]

# Step 1: Filter by read_contains_kmer_in_mcrs is True
read_df_fn = read_df_fn[read_df_fn['read_contains_kmer_in_mcrs']]

# Step 2: Filter by read_index between 10 and 140
read_df_fn = read_df_fn[(read_df_fn['read_index'] > 10) & (read_df_fn['read_index'] < 140)]

# Step 3: Filter by multimapped being False
read_df_fn = read_df_fn[read_df_fn['multimapped'] == False]

# Step 4: Join read_df with unique_mcrs_df on 'mcrs_header'
merged_df = read_df_fn.merge(unique_mcrs_df[['mcrs_header', 'number_of_kmers_with_overlap_to_other_mcrs_items_in_mcrs_reference', 'dlist', 'mcrs_sequence_length']], on='mcrs_header', how='left')

# Step 5: Filter by unique_mcrs_df conditions
read_df_fn = merged_df[
    (merged_df['number_of_kmers_with_overlap_to_other_mcrs_items_in_mcrs_reference'] == 0) &
    (merged_df['dlist'] == "none") &
    (merged_df['mcrs_sequence_length'] >= 2 * k - 1)
]

# Step 6: Drop any columns added during the merge that you don’t need in the final DataFrame
unexplainable_fn_reads = read_df_fn[read_df.columns]  # Keeps only the original read_df columns

print("Number of unexplainable FN reads: ", len(unexplainable_fn_reads))

In [ ]:
if os.path.exists(kb_extract_out_total):
    create_stratified_metric_bar_plot(read_df, 'mcrs_mutation_type', 'accuracy', overall_metric = metric_dictionary_reads['accuracy'], log_x_axis = False, display_numbers = True, out_path = f"{plot_output_folder}/accuracy_vs_mcrs_mutation_type_read_perspective.png")
    #* add more here

Make sense of the kb extract output
Update metrics now

In [ ]:
if run_kb_extract_slow:
    df_extract_combined = pd.DataFrame(columns=['read_header', 'mcrs_header_to_which_the_read_aligned'])

    # add some code here to check where these map
    for subdir in os.listdir(kb_extract_out_all_slow):
        subdir_path = os.path.join(kb_extract_out_all_slow, subdir)
        if os.path.isdir(subdir_path):
            fastq_file = os.path.join(subdir_path, '1.fastq.gz')
            if os.path.exists(fastq_file):
                # Run get_header_set_from_fastq for the fastq file
                aligned_reads_kb_extract = get_header_set_from_fastq(fastq_file)
                df_temp = pd.DataFrame(aligned_reads_kb_extract, columns=['read_header'])
                df_temp['mcrs_header_to_which_the_read_aligned'] = subdir  # Add subdirectory info
                df_extract_combined = pd.concat([df_extract_combined, df_temp], ignore_index=True)  # Concatenate the new DataFrame with df_extract_combined

    if not include_multimapped_reads:
        # remove multimapped reads
        df_extract_combined = df_extract_combined[~df_extract_combined['read_header'].isin(multimapped_reads)]
    else:
        # combines MCRSs for multimapped reads into a list - untested
        df_extract_combined = df_extract_combined.groupby('read_header')['mcrs_header_to_which_the_read_aligned'].apply(list).reset_index()


    read_df = read_df.merge(df_extract_combined, on='read_header', how='left')
    read_df['aligned_to_correct_mcrs'] = (read_df['mcrs_header'] == read_df['mcrs_header_to_which_the_read_aligned'])

    read_df.rename(columns={'TP': 'TP_crude', 'TN': 'TN_crude', 'FP': 'FP_crude', 'FN': 'FN_crude'}, inplace=True)

    read_df['TP'] = (read_df['mutant_read'] & read_df['aligned_to_correct_mcrs'])
    read_df['FP'] = (~read_df['mutant_read'] & read_df['aligned_to_correct_mcrs'])
    read_df['FN'] = (read_df['mutant_read'] & ~read_df['aligned_to_correct_mcrs'])
    read_df['TN'] = (~read_df['mutant_read'] & ~read_df['aligned_to_correct_mcrs'])

    metric_dictionary_reads_crude = metric_dictionary_reads.copy()
    metric_dictionary_reads = calculate_metrics(read_df, header_name = "read_header", check_assertions = check_assertions, out = f"{plot_output_folder}/read_metrics.txt")
    draw_confusion_matrix(metric_dictionary_reads)

    true_set = set(read_df.loc[read_df['mutant_read'], 'read_header'])
    positive_set = set(read_df.loc[read_df['aligned_to_correct_mcrs'], 'read_header'])
    create_venn_diagram(true_set, positive_set, TN = metric_dictionary_reads['TN'], mm = n_multimapped, out_path = f"{plot_output_folder}/venn_diagram_read.png")

## Perspective of reference items

Reference analysis

In [ ]:
unique_mcrs_df['TP'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['received_an_aligned_read'])
unique_mcrs_df['FP'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['received_an_aligned_read'])
unique_mcrs_df['FN'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['received_an_aligned_read'])
unique_mcrs_df['TN'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['received_an_aligned_read'])

metric_dictionary_reference = calculate_metrics(unique_mcrs_df, header_name = "mcrs_header", check_assertions = check_assertions, out = f"{plot_output_folder}/reference_metrics_crude.txt")
draw_confusion_matrix(metric_dictionary_reference)

In [ ]:
true_set = set(unique_mcrs_df.loc[unique_mcrs_df['included_in_synthetic_reads_mutant'], 'mcrs_header'])
positive_set = set(unique_mcrs_df.loc[unique_mcrs_df['received_an_aligned_read'], 'mcrs_header'])
create_venn_diagram(true_set, positive_set, TN = metric_dictionary_reference['TN'], out_path = f"{plot_output_folder}/venn_diagram_reference_crude.png")

In [ ]:
unexplainable_references = 0
for header in metric_dictionary_reference['FNs']:
    print(f"Header {header}")
    did_not_align_anywhere = (read_df[read_df['mcrs_header'] == header])['FN_crude'].all()
    print(f"All did not align anywhere for {header}: {did_not_align_anywhere}")
    did_not_align_to_correct_position = (read_df[read_df['mcrs_header'] == header])['FN'].all()
    print(f"All did not align to correct mcrs for {header}: {did_not_align_to_correct_position}")
    multimapped = (read_df[read_df['mcrs_header'] == header])['multimapped'].all()
    print(f"All multimapped for {header}: {multimapped}")
    if not did_not_align_anywhere and not did_not_align_to_correct_position and not multimapped:
        unexplainable_references += 1

if unexplainable_references > 0:
    print(f"Unexplainable references: {unexplainable_references}")
else:
    print("All explainable!")

In [ ]:
create_stratified_metric_bar_plot(unique_mcrs_df, 'number_of_reads_mutant', 'accuracy', overall_metric = metric_dictionary_reference['accuracy'], log_x_axis = False, out_path = f"{plot_output_folder}/accuracy_vs_number_of_reads_mutant.png")
#* add more here

#!!! create similar plots for y in {sensitivity, specificity}, and x in {number_of_reads_wt, tumor_purity} and determine cutoffs for which varseek is reliable

In [ ]:
aligned_read_df = read_df[read_df['aligned_somewhere'] == True]
mcrs_counts = aligned_read_df.groupby('mcrs_header').size().reset_index(name='number_of_reads_belonging_to_the_mcrs_item_that_aligned_somewhere')

unique_mcrs_df = unique_mcrs_df.merge(mcrs_counts, on='mcrs_header', how='left')
unique_mcrs_df['number_of_reads_belonging_to_the_mcrs_item_that_aligned_somewhere'] = unique_mcrs_df['number_of_reads_belonging_to_the_mcrs_item_that_aligned_somewhere'].fillna(0).astype(int)
unique_mcrs_df['number_of_reads_belonging_to_the_mcrs_item_that_did_not_align_anywhere'] = unique_mcrs_df['number_of_reads_mutant'] - unique_mcrs_df['number_of_reads_belonging_to_the_mcrs_item_that_aligned_somewhere']

mutant_mask = unique_mcrs_df['included_in_synthetic_reads_mutant'] == True
wt_mask = unique_mcrs_df['included_in_synthetic_reads_wt'] == True

# Perform the calculation only on the selected rows
unique_mcrs_df.loc[mutant_mask, 'number_of_reads_belonging_to_the_mcrs_item_that_did_not_align_anywhere'] = (
    unique_mcrs_df.loc[mutant_mask, 'number_of_reads_mutant'] - unique_mcrs_df.loc[mutant_mask, 'number_of_reads_belonging_to_the_mcrs_item_that_aligned_somewhere']
)

unique_mcrs_df.loc[wt_mask, 'number_of_reads_belonging_to_the_mcrs_item_that_did_not_align_anywhere'] = (
    unique_mcrs_df.loc[wt_mask, 'number_of_reads_wt'] - unique_mcrs_df.loc[wt_mask, 'number_of_reads_belonging_to_the_mcrs_item_that_aligned_somewhere']
)

unique_mcrs_df['mutation_expression_prediction_error'] = unique_mcrs_df['number_of_reads_aligned_to_this_item'] - unique_mcrs_df['number_of_reads_mutant']  # positive means overpredicted, negative means underpredicted

In [ ]:
if run_kb_extract_slow:
    # Step 1: Group by 'mcrs_header' and count 'aligned_to_correct_mcrs' (sum will count True as 1, False as 0)
    aligned_counts = read_df.groupby('mcrs_header')['aligned_to_correct_mcrs'].sum().reset_index()

    # Rename the column to something meaningful, like 'correct_alignment_count'
    aligned_counts.rename(columns={'aligned_to_correct_mcrs': 'number_of_reads_belonging_to_this_mcrs_item_that_mapped_here_correctly'}, inplace=True)

    # Step 2: Merge the counts with unique_mcrs_df on the 'mcrs_header' column
    unique_mcrs_df = pd.merge(unique_mcrs_df, aligned_counts, on='mcrs_header', how='left')

    # Fill NaN values with 0 in case some mcrs_header values in unique_mcrs_df do not appear in read_df
    unique_mcrs_df['number_of_reads_belonging_to_this_mcrs_item_that_mapped_here_correctly'].fillna(0, inplace=True)

    unique_mcrs_df['received_an_aligned_read_from_one_of_its_corresponding_reads'] = unique_mcrs_df['number_of_reads_belonging_to_this_mcrs_item_that_mapped_here_correctly'] > 0

In [ ]:
if run_kb_extract_slow:
    # Step 1: Group by 'mcrs_header_to_which_the_read_aligned' and collect 'read_header' into lists
    reads_mapped = read_df.groupby('mcrs_header_to_which_the_read_aligned')['read_header'].apply(list).reset_index()

    # Rename the column to 'reads_mapped_to_this_reference_item' for clarity
    reads_mapped.rename(columns={'read_header': 'reads_mapped_to_this_reference_item'}, inplace=True)

    # Step 2: Merge the list of read headers with unique_mcrs_df on the 'mcrs_header' column
    unique_mcrs_df = pd.merge(unique_mcrs_df, reads_mapped, left_on='mcrs_header', right_on='mcrs_header_to_which_the_read_aligned', how='left')
    unique_mcrs_df.drop(columns='mcrs_header_to_which_the_read_aligned', inplace=True)

    # Fill NaN values with empty lists in case there are no matching read headers for some mcrs_header values
    unique_mcrs_df['reads_mapped_to_this_reference_item'] = unique_mcrs_df['reads_mapped_to_this_reference_item'].apply(lambda x: x if isinstance(x, list) else [])

Improved stats

In [ ]:
if run_kb_extract_slow:
    unique_mcrs_df['number_of_reads_belonging_to_the_mcrs_item_that_aligned_somewhere'] = unique_mcrs_df['number_of_reads_belonging_to_the_mcrs_item_that_aligned_somewhere'].astype(bool)

    unique_mcrs_df.rename(columns={'TP': 'TP_crude', 'TN': 'TN_crude', 'FP': 'FP_crude', 'FN': 'FN_crude'}, inplace=True)

    unique_mcrs_df['TP'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['received_an_aligned_read_from_one_of_its_corresponding_reads'])
    unique_mcrs_df['FP'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & unique_mcrs_df['received_an_aligned_read_from_one_of_its_corresponding_reads'])
    unique_mcrs_df['FN'] = (unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['received_an_aligned_read_from_one_of_its_corresponding_reads'])
    unique_mcrs_df['TN'] = (~unique_mcrs_df['included_in_synthetic_reads_mutant'] & ~unique_mcrs_df['received_an_aligned_read_from_one_of_its_corresponding_reads'])

    metric_dictionary_reference = calculate_metrics(unique_mcrs_df, header_name = "mcrs_header", check_assertions = check_assertions, out = f"{plot_output_folder}/reference_metrics.txt")
    draw_confusion_matrix(metric_dictionary_reference)

    true_set = set(unique_mcrs_df.loc[unique_mcrs_df['included_in_synthetic_reads_mutant'], 'mcrs_header'])
    positive_set = set(unique_mcrs_df.loc[unique_mcrs_df['received_an_aligned_read_from_one_of_its_corresponding_reads'], 'mcrs_header'])
    create_venn_diagram(true_set, positive_set, TN = metric_dictionary_reference['TN'], out_path = f"{plot_output_folder}/venn_diagram_reference.png")

In [ ]:
if run_kb_extract_slow:
    unique_mcrs_df['number_of_reads_belonging_to_this_mcrs_item_that_did_not_align_correctly'] = unique_mcrs_df['number_of_reads_mutant'] - unique_mcrs_df['number_of_reads_belonging_to_this_mcrs_item_that_mapped_here_correctly']
    unique_mcrs_df['number_of_reads_aligned_to_this_mcrs_item_that_aligned_incorrectly'] = unique_mcrs_df['number_of_reads_aligned_to_this_item'] - unique_mcrs_df['number_of_reads_belonging_to_this_mcrs_item_that_mapped_here_correctly']

In [ ]:
plot_histogram(unique_mcrs_df.loc[~unique_mcrs_df['TN']], 'mutation_expression_prediction_error', log_scale = False, out_path = f"{plot_output_folder}/histogram_mutation_expression_prediction_error_linear_axis.png")
plot_histogram(unique_mcrs_df.loc[~unique_mcrs_df['TN']], 'mutation_expression_prediction_error', log_scale = True, out_path = f"{plot_output_folder}/histogram_mutation_expression_prediction_error_log_axis.png")

In [ ]:
metric_dictionary_reference['mutation_expression_prediction_error_abs_mean'] = unique_mcrs_df.loc[~unique_mcrs_df['TN'], 'mutation_expression_prediction_error'].abs().mean()
create_stratified_metric_bar_plot(unique_mcrs_df, 'number_of_reads_mutant', 'expression_error', overall_metric = metric_dictionary_reference['mutation_expression_prediction_error_abs_mean'], log_x_axis = False, out_path = f"{plot_output_folder}/expression_error_vs_number_of_reads_mutant.png")
create_stratified_metric_bar_plot(unique_mcrs_df, 'distance_to_nearest_splice_junction', 'accuracy', overall_metric = metric_dictionary_reference['accuracy'], log_x_axis = False, bins = [-1, 0, 1, 5, 10, 50, 100, float('inf')])

Save dfs

In [ ]:
unique_mcrs_df_out_path = f"{synthetic_reads_parent}/unique_mcrs_df.csv"
unique_mcrs_df.to_csv(unique_mcrs_df_out_path, index=False)

read_df_out_path = f"{synthetic_reads_parent}/read_df.csv"
read_df.to_csv(read_df_out_path, index=False)

In [ ]:
print(read_df.columns)
print(unique_mcrs_df.columns)
print(mutation_metadata_df.columns)

In [ ]:
# no_noise_read_metrics = ""
# substitution_noise_read_metrics = ""
# deletion_noise_read_metrics = ""
# insertion_noise_read_metrics = ""
# all_noise_read_metrics = ""
# metric = "accuracy"



# accuracy_dict = {}
# accuracy_dict['No Noise'] = retrieve_value_from_metric_file(metric, no_noise_read_metrics)
# accuracy_dict['Substitution Noise'] = retrieve_value_from_metric_file(metric, substitution_noise_read_metrics)
# accuracy_dict['Deletion Noise'] = retrieve_value_from_metric_file(metric, deletion_noise_read_metrics)
# accuracy_dict['Insertion Noise'] = retrieve_value_from_metric_file(metric, insertion_noise_read_metrics)
# accuracy_dict['All Noise'] = retrieve_value_from_metric_file(metric, all_noise_read_metrics)

# from matplotlib import pyplot as plt

plot_basic_bar_plot_from_dict(accuracy_dict, y_axis=metric)